In [ ]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import cuml 
import cupy as cp
import cudf as cd
import seaborn as sns

In [ ]:
df = cd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
null_column = []
for col in df.columns:
    if df[col].isnull().sum()>=1:
        null_column.append(col)

In [ ]:
df_train = df.drop(null_column,axis = 1)

In [ ]:
df_train.head(4)

In [ ]:
df_train.info()

In [ ]:
final_df = df_train[['OverallQual', 'GrLivArea', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF', 'FullBath']]
final_df.head()

In [ ]:
X = final_df
Y = df_train['SalePrice']


In [ ]:
X

In [ ]:
from cuml.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_scaled.as_matrix(),Y,test_size = 0.25,random_state = 42)

In [ ]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']

In [ ]:
from cuml import LinearRegression
err = 9999999999
final_lr = None
for alg in algorithm:
    print("Algorithm:")
    print(alg)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = alg)
    reg = lr.fit(X_train,Y_train)
    preds = lr.predict(X_test)
    Y_test=Y_test.astype(cp.float64)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
    if err>=cuml.metrics.regression.mean_squared_error(Y_test,preds):
        err = cuml.metrics.regression.mean_squared_error(Y_test,preds)
        final_lr = lr
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y_test,preds))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

In [ ]:
test = cd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test.head()
test_id = test[['Id']]

In [ ]:
test = test[['OverallQual', 'GrLivArea', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF', 'FullBath']]
test=test.fillna(0)
test.head()

In [ ]:
pred = final_lr.predict(test)

In [ ]:
type(test_id)
test_id = test_id.to_pandas()

In [ ]:
type(pred)

In [ ]:
import pandas as pd
output = pd.DataFrame()

In [ ]:
output["Id"] = test_id

In [ ]:
output["SalePrice"] = pd.DataFrame(pred)

In [ ]:
output.to_csv('submission.csv', index=False)